# Pixel sampling

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import starry
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import pymc3 as pm
import exoplanet
import theano
import theano.tensor as tt

starry.config.lazy = True
starry.config.quiet = True
cmap = plt.get_cmap("plasma")
cmap.set_under("#333333")
cmap.set_over("w")
cnorm = lambda: colors.Normalize(vmin=0.0)

## Mock data

### Mock surface map

In [ ]:
# Params
ydeg_tru = 20
ydeg_inf = 10
inc = 85

In [ ]:
map_tru = starry.Map(ydeg_tru, inc=inc)
map_tru.add_spot(amp=-0.03, relative=False, sigma=0.05, lat=30, lon=0)
map_tru.add_spot(amp=-0.06, relative=False, sigma=0.1, lat=-20, lon=60)
map_tru.add_spot(amp=-0.03, relative=False, sigma=0.05, lat=10, lon=150)
map_tru.add_spot(amp=-0.03, relative=False, sigma=0.05, lat=60, lon=-90)
map_tru.add_spot(amp=-0.025, relative=False, sigma=0.04, lat=-30, lon=-90)
map_tru.add_spot(amp=-0.025, relative=False, sigma=0.04, lat=0, lon=-150)
map_tru.amp = 1.0
y0 = np.array(map_tru.y.eval())
map_tru.show(projection="moll", colorbar=True, norm=cnorm(), cmap=cmap)

### Mock light curve

In [ ]:
# Params
prot = 1.0 / 7.0
bo = [-0.5, 0.25, 0.75, 0.5]
ro = [0.5, 0.75, 0.3, 0.2]
time = np.linspace(0, 1, 1000)
theta = (360.0 * time / prot).reshape(len(ro), -1)
X_tru = np.vstack(
    [
        map_tru.design_matrix(
            theta=theta[i],
            xo=np.linspace(-1 - ro[i], 1 + ro[i], len(theta[i])),
            yo=bo[i],
            ro=ro[i],
        ).eval()
        for i in range(4)
    ]
)

In [ ]:
# Generate the light curve
flux_tru = X_tru.dot(y0)

# Add noise
np.random.seed(0)
ferr_tru = 1e-3
flux = flux_tru + ferr_tru * np.random.randn(len(flux_tru))

# Normalize
norm = np.nanmedian(flux)
flux = flux / norm
ferr = ferr_tru / norm

# Plot
plt.plot(time, flux, "k.", alpha=0.3, label="observed")
plt.plot(time, flux_tru / norm, "C0", label="true")
plt.legend()
plt.xlabel("time")
plt.ylabel("flux");

## Inference

Pre-compute some linear operators:

In [ ]:
map_inf = starry.Map(ydeg_inf, inc=inc)
X_inf = X_tru[:, : (ydeg_inf + 1) ** 2]
lat, lon, Y2P, P2Y, Dx, Dy = map_inf.get_pixel_transforms(oversample=4)
npix = lat.shape[0]

Define a function to compare the result to the true map:

In [ ]:
def plot(y, amp, flux_model):

    fig = plt.figure(figsize=(8, 8))
    fig.subplots_adjust(wspace=0.15)
    ax = [
        plt.subplot2grid((3, 2), (0, 0), colspan=1, rowspan=1),
        plt.subplot2grid((3, 2), (0, 1), colspan=1, rowspan=1),
        plt.subplot2grid((3, 2), (1, 0), colspan=2, rowspan=1),
        plt.subplot2grid((3, 2), (2, 0), colspan=2, rowspan=1),
    ]

    # Show the true map
    map_tru.show(
        ax=ax[0], projection="moll", colorbar=True, norm=cnorm(), cmap=cmap,
    )

    # Show the inferred map
    map_inf.amp = amp
    map_inf[1:, :] = y[1:]
    map_inf.show(
        ax=ax[1], projection="moll", colorbar=True, norm=cnorm(), cmap=cmap,
    )

    # Show the flux model
    ax[2].plot(time, flux, "k.", alpha=0.3, label="observed")
    ax[2].plot(time, flux_model, "C1", label="model")
    ax[2].legend(fontsize=10, loc="lower right")
    ax[2].set_xlabel("time")
    ax[2].set_ylabel("flux")

    # Show the pixel distributions
    pix_tru = map_tru.render(projection="moll").eval().flatten()
    pix_tru /= np.nanmax(pix_tru)
    pix_tmp = map_inf.render(projection="moll").eval().flatten()
    pix_tmp /= np.nanmax(pix_tmp)
    ax[3].hist(pix_tru, bins=50, alpha=0.5, label="true")
    ax[3].hist(pix_tmp, bins=50, alpha=0.5, label="inferred")
    ax[3].legend()
    ax[3].set_yticks([])
    ax[3].set_ylabel("pixels")
    ax[3].set_xlabel("intensity");

### L2 inference

In [ ]:
map_inf.set_data(flux, C=ferr ** 2)
map_inf.set_prior(L=1e-2)
map_inf.solve(design_matrix=X_inf)
y_L2 = np.array(map_inf.y.eval())
amp_L2 = map_inf.amp.eval()
flux_L2 = amp_L2 * X_inf.dot(y_L2)

In [ ]:
plot(y_L2, amp_L2, flux_L2)

### Pixel inference w/ uniform prior

In [ ]:
with pm.Model(theano_config=dict(compute_test_value="ignore")) as model:

    # Uniform prior on the *pixels*
    p = pm.Uniform("p", lower=0.0, upper=1.0, shape=(npix,))
    x = tt.dot(P2Y, p)

    # Compute the flux
    flux_model = tt.dot(X_inf, x)
    pm.Deterministic("flux_model", flux_model)
    flux_model_guess = exoplanet.eval_in_model(flux_model)

    # Store the Ylm coeffs
    pm.Deterministic("amp", x[0])
    pm.Deterministic("y", x / x[0])

    # The likelihood function assuming known Gaussian uncertainty
    pm.Normal("obs", mu=flux_model, sd=ferr, observed=flux)

In [ ]:
with model:
    soln = exoplanet.optimize(options=dict(maxiter=9999))
    y_pix = np.array(soln["y"])
    amp_pix = soln["amp"]
    flux_pix = soln["flux_model"]

In [ ]:
plot(y_pix, amp_pix, flux_pix)

## Pixel inference w/ Beta prior

In [ ]:
with pm.Model(theano_config=dict(compute_test_value="ignore")) as model:

    # Beta prior on the *pixels*
    p = pm.Beta("p", alpha=0.5, beta=0.5, shape=(npix,))
    norm = pm.Normal("norm", mu=0.5, sd=0.25)
    x = norm * tt.dot(P2Y, p)

    # Compute the flux
    flux_model = tt.dot(X_inf, x)
    pm.Deterministic("flux_model", flux_model)
    flux_model_guess = exoplanet.eval_in_model(flux_model)

    # Store the Ylm coeffs
    pm.Deterministic("amp", x[0])
    pm.Deterministic("y", x / x[0])

    # The likelihood function assuming known Gaussian uncertainty
    pm.Normal("obs", mu=flux_model, sd=ferr, observed=flux)

In [ ]:
with model:
    soln = exoplanet.optimize(options=dict(maxiter=9999))
    y_beta = np.array(soln["y"])
    amp_beta = soln["amp"]
    flux_beta = soln["flux_model"]

In [ ]:
plot(y_beta, amp_beta, flux_beta)

## Pixel inference w/ TV prior

In [ ]:
with pm.Model(theano_config=dict(compute_test_value="ignore")) as model:

    # Uniform prior on the *pixels*
    p = pm.Uniform("p", lower=0.0, upper=1.0, shape=(npix,))
    x = tt.dot(P2Y, p)

    # Apply the TV penalty
    # TODO: marginalize over theta w/ a suitable prior?
    theta = 0.05
    TV = tt.sum(tt.abs_(tt.dot(Dx, p)) + tt.abs_(tt.dot(Dy, p)))
    pm.Potential("TV", -TV / theta)

    # Compute the flux
    flux_model = tt.dot(X_inf, x)
    pm.Deterministic("flux_model", flux_model)
    flux_model_guess = exoplanet.eval_in_model(flux_model)

    # Store the Ylm coeffs
    pm.Deterministic("amp", x[0])
    pm.Deterministic("y", x / x[0])

    # The likelihood function assuming known Gaussian uncertainty
    pm.Normal("obs", mu=flux_model, sd=ferr, observed=flux)

In [ ]:
with model:
    soln = exoplanet.optimize(options=dict(maxiter=9999))
    y_tv = np.array(soln["y"])
    amp_tv = soln["amp"]
    flux_tv = soln["flux_model"]

In [ ]:
plot(y_tv, amp_tv, flux_tv)

## Compare all the methods

In [ ]:
fig, ax = plt.subplots(5, figsize=(5, 10))
map_tru.show(
    ax=ax[0], projection="moll", colorbar=True, norm=cnorm(), cmap=cmap,
)
ax[0].annotate(
    "true",
    xy=(-2 * np.sqrt(2), np.sqrt(2)),
    xycoords="data",
    ha="left",
    va="top",
    fontsize=10,
    fontweight="bold",
)
for i, amp, y, label in zip(
    [1, 2, 3, 4],
    [amp_L2, amp_pix, amp_beta, amp_tv],
    [y_L2, y_pix, y_beta, y_tv],
    ["L2", "U", "Beta", "TV"],
):
    map_inf.amp = amp
    map_inf[1:, :] = y[1:]
    map_inf.show(
        ax=ax[i], projection="moll", colorbar=True, norm=cnorm(), cmap=cmap,
    )
    ax[i].annotate(
        label,
        xy=(-2 * np.sqrt(2), np.sqrt(2)),
        xycoords="data",
        ha="left",
        va="top",
        fontsize=10,
        fontweight="bold",
    )